Импортируем нужные для работы библиотеки

In [1]:
! pip install spacy
! python -m spacy download ru_core_news_sm

                                              0.0/15.3 MB ? eta -:--:--
                                              0.0/15.3 MB 1.3 MB/s eta 0:00:12
                                              0.1/15.3 MB 1.1 MB/s eta 0:00:15
                                              0.2/15.3 MB 1.2 MB/s eta 0:00:13
                                              0.2/15.3 MB 1.2 MB/s eta 0:00:13
                                              0.3/15.3 MB 1.5 MB/s eta 0:00:11
     -                                        0.4/15.3 MB 1.7 MB/s eta 0:00:09
     -                                        0.5/15.3 MB 1.8 MB/s eta 0:00:08
     -                                        0.6/15.3 MB 1.9 MB/s eta 0:00:08
     -                                        0.7/15.3 MB 2.0 MB/s eta 0:00:08
     --                                       0.8/15.3 MB 2.1 MB/s eta 0:00:07
     --                                       0.9/15.3 MB 2.1 MB/s eta 0:00:07
     --                                       1.0/15.3 MB 2

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json

# Предобработка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)

,Unnamed: 0,status,url,text,likes,reposts,comments
0,0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0
1,1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0
2,2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11238 non-null  int64  
 1   status      11238 non-null  object 
 2   url         11238 non-null  object 
 3   text        11238 non-null  object 
 4   likes       11081 non-null  object 
 5   reposts     10908 non-null  float64
 6   comments    10902 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 614.7+ KB


In [5]:
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем данные по количеству символов в постах. Оставим посты только длиннее 100 символов.

In [6]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Для обработки нам нужны только посты на кириллице. Отфильтруем посты содержащие кириллицу.

In [7]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты в столбце с текстами

In [8]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка текстов

In [9]:
model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

Складываем все посты в один список

In [10]:
texts = data_dd["text"].tolist()

Выполняем лемматизацию текстов. Наберитесь терпения - это долгий процесс, который может занять около часа.

In [11]:
if os.path.exists('lemmas.json'):
    with open("lemmas.json") as f:
        res = json.load(f)['lemmas']
else:
    res = []
    for doc in model.pipe(texts, disable=["tagger", "parser"]):
        res.append([token.lemma_ for token in doc])

    with open("lemmas.json", "w") as fid:
        json.dump({"lemmas": res}, fid)

Смотрим на результат - случайным образом сэмплируем несколько примеров

In [12]:
", ".join(res[np.random.randint(len(res))])

'разбирать, сегодня, на, карьерный, консультация, с, клиент, переход, из, менеджерский, роль, с, технический, -, в, it, ., как, и, многие, сейчас, ,, клиент, решить, ,, что, сфера, it, наиболее, перспективный, и, открывать, более, лёгкий, возможность, для, удалённый, работа, из, любой, удобный, точка, с, наличие, крепкий, wi, -, fi, (, а, у, клиент, мечта, -, перевезти, семья, через, пара, год, близкий, к, морю).что, сделать, :, ✅, поработали, с, резюме, :, упаковали, предыдущий, опыт, ,, выделить, из, него, полезный, навык, для, будущий, роль, ,, сделать, более, функциональный, и, информативный, с, точка, зрение, профиль, ;, ✅, добавить, пройти, хороший, обучение, по, специализация, и, пройденные, тестовый, задание, в, качество, начальный, портфолио, ;, ✅, передать, внушительный, список, сайт, ,, канал, для, самостоятельный, отслеживание, отраслевой, вакансия, и, публикация, резюме, ;, ✅, дать, задание, на, анализ, рынок, компания, -, клиент, интересный, определённый, направление, в, 

In [13]:
", ".join(res[np.random.randint(len(res))])

'опенспейсы, -, одна, из, главный, причина, ,, по, которой, я, предпочитать, только, удалить, работа, ., Конечно, ,, не, единственный, ,, также, я, любить, экономить, время, на, дорога, и, жить, там, ,, где, мне, хотеться, ,, а, не, близкий, к, офис, ., быть, ещё, куча, фактор, ,, по, которым, удалённый, работа, намного, хороший, офисный, ,, но, я, не, видеть, ни, одного, плюс, у, офис, ., весь, этот, аргумент, ,, наподобие, :, общение, с, коллега, и, прочие, формулировка, ,, относиться, к, коммуникация, не, являться, весомымм, аргумент, в, современный, мир, ,, где, быть, интернет, ,, куча, мессенджер, и, прочего, ., да, и, согласиться, ,, письмо, куда, крутой, ,, чем, разговор, ., в, первый, ,, никакой, потеря, время, на, междометие, и, прочий, звуковой, мусор, ., в, второй, ,, когда, писать, -, чёткий, формулировать, свой, мысль, ., а, в, третий, ,, в, любой, момент, можно, перечитать, написанное, ., удалить, сотрудник, также, выгодный, и, работодатель, ., не, нужный, тратить, деньга

Удалим знаки препинания, цифры и слова, содержащие цифры. Для это соберём регулярное выражение, которое подходит для любого слова, состоящего из букв кирилицы.

In [14]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

corpus = list(map(remove_symbols, res))

In [15]:
", ".join(corpus[np.random.randint(len(corpus))])

'популярный, вид, сайт, для, бизнес, одностраничный, сайт, нужный, для, того, чтобы, быстро, конвертировать, клиент, одним, из, вид, такой, сайт, целевой, назначение, которого, превратить, обычный, посетитель, в, покупатель, подписчик, и, побудить, его, к, какой, либо, конкретный, действие, подписаться, на, рассылка, или, просто, зарегистрироваться, на, сайт, скачать, прайс, или, книга, ознакомить, клиент, с, акция, или, мероприятие, и, пробудить, в, нем, желание, прийти, именно, к, вам, информация, на, такой, сайт, подаваться, емко, красочно, броско, метод, уникальный, дизайн, сайт, визитка, знакомить, потенциальный, клиент, с, ваш, компания, ваш, товар, или, услуга, и, предоставлять, связь, клиент, с, вами, у, этот, сайт, нет, цель, сделать, какой, то, супер, популярный, площадка, в, среднее, сайт, визитка, занимать, всего, страница, он, содержать, информация, о, компания, товар, услуга, местонахождение, блог, статья, полный, статья'

# Векторизация текстов

Теперь необходимо перевести тексты новостей и слова, из которых они состоят, в числовое представление, т.е. провести векторизацию. Для этого можно использовать подход "мешок слов" (Bag of Words) или Tf-iDf. Второй вариант нравится мне больше.

Чтобы уменьшить размерность получаемой матрицы, добавим исключение стоп-слов.

In [16]:
stop_words = stopwords.words('russian')
print(stop_words)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [17]:
tfidf = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.9)

Т.е. TfIdfVectorizer принимает на вход список предложений, поэтому необходимо объединить списки слов, составляющих корпус, в предложения.

In [18]:
docs = [" ".join(tokens) for tokens in corpus]
docs[:3]

['всем свой первый статья на про история и технология разработка игра и как мы создать игра с элемент машинный обучение и выйти в ноль за месяцапри внимательный прочтение вы даже смочь запустить первый версия игра',
 'если вам интересный позалипать в слово я запустить понравиться брат шникам',
 'с трепет готовый представить свой продукт рефни первый сервис реферального рекрутинг в ит на нем каждый мочь посоветовать знакомого на открытый позиция в компания и получить за рекомендация денежный вознаграждение идея создание внешний реферального сервис зародиться у меня давно в период пандемия я посоветовать пара человек на хороший должность в разный компания их взять все счастливый тогда я и подумать пора создание команда подготовка тз изучение самых разный нюанс о которых я ранний не знать и самое волнительный поиск инвестор все получиться и весь это теперь позади а впереди только постоянный развитие продукт посмотреть что у нас выйти быть благодарный если вы поддержать меня и мой команда 

In [19]:
x = tfidf.fit_transform(docs)

In [20]:
x.shape

(1862, 1635)

Теперь составим словарь {id_токена: токен} - он пригодится нам позднее.

In [21]:
id2word = {i: token for i, token in enumerate(tfidf.get_feature_names_out())}

# LDA

Теперь можем запустить алгоритм LDA. Зададим количество тем равное 20.

In [22]:
N_TOPICS = 10

In [23]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=765)

In [24]:
topics = lda.fit_transform(x)

In [25]:
topics.shape

(1862, 10)

Получили массив размера 1862x20. Строки здесь - отдельные тексты, а колонки - 20 выделенных тем. В каждой ячейке стоит вероятность того, что данный текст относится к данной теме. Это то самое распределение тем над документами.

## Ключевые слова

Теперь извлечём ключевые слова для каждой из 20 тем. В свойсте components _ лежит матрица соответствия слов темам. Пройдёмся по каждой из 20 выделенных тем и для каждой возьмём ТОП-20 слов.

По очереди пройдёмся по каждой теме (первое измерение массиа) и выберем ТОП-20 слов, наиболее характерных для этой темы. Потом по индексу слова обратимся к словарю и узнаем, что это за слово.

In [26]:
for i in range(N_TOPICS):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
задача, сделать, новый, хотеть, курс, делать, работать, первый, которые, нужный, вопрос, очень, год, время, весь, работа, человек, мочь, свой, это

 
Тема 2 :
буде, ваш, може, чи, свой, ви, або, якщо, проект, можна, але, роботи, також, день, ми, це, час, наш, як, що

 
Тема 3 :
приложение, возможность, всем, привет, знание, специалист, продукт, новый, бизнес, год, искать, вакансия, разработчик, наш, компания, разработка, проект, опыт, работа, команда

 
Тема 4 :
новый, поддержка, карьерный, совет, предложить, поиск, рекомендация, контакт, ваш, заранее, здравствовать, благодарить, хотеть, возможность, пройти, следующий, проверка, значок, тест, навык

 
Тема 5 :
рассказать, коллега, декабрь, бот, движение, вебинар, технология, приглашать, достижение, зп, кейс, мир, рассказывать, форум, обеспечение, кибербезопасность, инфобез, сегодня, иб, добрый

 
Тема 6 :
заходить, приводить, сайт, программа, яндекс, стажировка, стартовать, обновить, управлениепроектами, погнать, конверсия, ле

По полученным ключевым словам можно составить представление о содержании той или иной темы. Хотя иногда сочетания слов довольно странные )

## Типичные статьи

Теперь для каждой темы возьмём по одному документу, который хорошо соответсвует этой теме. Для этого заглянем в массив topics.

In [27]:
for i in range(N_TOPICS):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i)
    print(data_dd.iloc[doc_id]["text"])
    print("\n")

Тема  0
Всем привет.Сегодняшняя тема навеяна книгой Барбары Шер, о которой писал ранее “О чем мечтать”.Осторожно, затрагиваются психологические аспекты, а не только карьерные ❗️Особый тип людей - бунтари против обыденности часть 1. 💪История из моей жизни.У меня есть знакомые, у которых очень точные и понятные мечты – “Хочу яхту и быть ее капитаном” ☄️, “Я прекрасный (дизайнер, конструктор, просто человек) и хочу, что бы мир дал мне то, что мне причитается (богатство, славу, признание)” 💥 и “Обычная работа это не для меня”. 🌞Я никогда не понимал такого, и с ходу задавал вопросы типа ❓ – А что ты делаешь что бы приблизиться к своей мечте ? ❓Обычно, в ответ я получал очень говорящий взгляд 😡 и продолжение описание, какая будет классная жизнь, когда все это случиться. 🌈Я не мог понять почему, люди игнорируют такую простую причинно-следственную связь, ответ нашелся, но все по порядку.У подобной мечты и типа поведения есть ряд подводных камней:- все не самое лучшее кажется бессмысленным 😫- п